In [1]:
import numpy as np
import pandas as pd
import requests

from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans


In [18]:
# Import Data
print('Load Description Embeddings')
desc_emb = np.load('../data/desc_emb.npz')
desc_embeddings = dict(zip(("desc_embeddings"), (desc_emb[k] for k in desc_emb)))['d']
print(desc_embeddings.shape)

print('Load Description Labels, Address Labels and Name Labels')
NFT_attributes = np.load('../data/NFT_attributes.npz')
d = dict(zip(("description_list","contract_addresses","name_list"), (NFT_attributes[k] for k in NFT_attributes)))
description_list = d['description_list']
contract_addresses = d['contract_addresses']
name_list = d['name_list']
print(description_list.shape)

Load Description Embeddings
(11083, 768)
Load Description Labels, Address Labels and Name Labels
(11083,)


In [19]:
contract_addresses[0:20]

array(['0x14232931f4151a7a962c1929F670378159d87fb7',
       '0xC4eaEAfa1E796a2a0E92aDA206499E0c9e9a3B3a',
       '0x8E34450a3C674d47B6494f346Ac3D67f8c8dfc15',
       '0x7a414ECfcD7DAd240B1E6dD694a60F956425967e',
       '0xC52877e8b234307f2bA61AC0b0Eb64e5DBA0719f',
       '0x670a9AfE31eb88543657bd91cD423FE9Bd3a5817',
       '0xea185bBFD484dc5690220771583ceeff4309F46C',
       '0x31445321df7C12d81E196c3821828D7Cf5763DE7',
       '0x129C3727E8CC24D513Bc5F8B87616e1754b64aCd',
       '0x39E02d4Ce3d8E727818a07Edb67220423D223578',
       '0xf5164b2626e0f3e96cb4210a27C46fFb2afb5CC7',
       '0x349d140E002a4149D34cd82fB3c5E02446B7B6a5',
       '0x67b16a95D09ba7A08a65cf4674222212925c7e19',
       '0x1E08D77d2F7769D47007D52b6c119b474Da6174D',
       '0x66a4A1227b471e709eE76303eCE790820cad55cc',
       '0xAE2ca931225d4d6C382AfBC147dfDC2835E713Db',
       '0xcE22F9999838bF042D8a8fF4D8FAEeaFdA2F9649',
       '0x41D265c6095ADF003c4f0141583798A7119E52d7',
       '0x41e244a2b368B04F07187819dbBA93f9a648

In [9]:
# Get Owner Adress + NFTs
OWNER = '0xe11BFCBDd43745d4Aa6f4f18E24aD24f4623af04'

URL = f'https://eth-mainnet.alchemyapi.io/v2/demo/getNFTs/?owner={OWNER}'
Owner_NFTs = requests.get(URL).json()['ownedNfts']

print(Owner_NFTs[0]['contract']['address'])

owner_contract_addresses = []
token_ids = []
for nft in Owner_NFTs:
    owner_contract_addresses.append(nft['contract']['address'])
    token_ids.append(nft['id']['tokenId'])



0x000386e3f7559d9b6a2f5c46b4ad1a9587d59dc3


In [ ]:

attempts = 0
successes = 0
for address in contract_addresses:
    attempts += 1
    URL = f"https://eth-mainnet.alchemyapi.io/v2/demo/getNFTMetadata?contractAddress={address}&tokenId=1&tokenType=erc721"
    NFT = requests.get(URL)
    try:
        NFTjson = NFT.json()
    except:
        continue
    if (type(NFTjson) is dict and "media" in NFTjson.keys() and type(NFTjson["media"]) is dict and "gateway" in NFTjson["media"].keys()):
        successes += 1
        image = NFTjson["metadata"]["gateway"]
        print('Raw:', image)
    print("Attempts: ", attempts, " Successes: ", successes)

In [12]:
# Get NFT Description
Owner_descriptions = []
Owner_available_addresses = []
Owner_names = []

for address in contract_addresses:
    URL = f"https://eth-mainnet.alchemyapi.io/v2/demo/getNFTMetadata?contractAddress={address}&tokenId=1&tokenType=erc721"
    NFT = requests.get(URL)
    try:
        NFTjson = NFT.json()
    except:
        continue
    if (type(NFTjson) is dict and "metadata" in NFTjson.keys() and type(NFTjson["metadata"]) is dict and "description" in NFTjson["metadata"].keys()):
        description = NFTjson["metadata"]["description"]
        Owner_descriptions.append(description)
        Owner_available_addresses.append(address)
        print('Desc:', description)
        name = "Not Available"
        if (type(NFTjson["metadata"]) is dict and "name" in NFTjson["metadata"].keys()):
            name = NFTjson["metadata"]["name"]
        Owner_names.append(name)

print(Owner_descriptions[0])

Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [https://nikeape.xyz](https://nikeape.xyz)
Desc: COUNTDOWN OVER. MINTING LIVE.  [ht

In [ ]:
print('Loading Model')
model = SentenceTransformer('bert-base-nli-mean-tokens')

#description embeddings
print(f'Calculating embeddings for {len(Owner_descriptions)} descriptions')
Owner_desc_embeddings = model.encode(Owner_descriptions, normalize_embeddings=True, show_progress_bar=True, batch_size = 64)

In [ ]:
print('Calculating Recommendations')

recommendatons = []

for i, owner_emb in enumerate(Owner_desc_embeddings):
    min_index = 0
    min_distance = 10

    for j, dj in enumerate(desc_embeddings):
        distance = np.sum(np.square((owner_emb - dj)))
        if distance < min_distance and distance > 1e-6:
            min_distance = distance
            min_index = j
    recommendatons.append(min_index)
    print(f'The closest NFT for Owner_NFT_{i} is All_NFT_{min_index} with a distance of {min_distance}')
    print(f'\ndescription Owner_NFT_{i}: {Owner_descriptions[:100]}')
    print(f'\ndescription All_NFT_{min_index}: {description_list[min_index][:100]}')
    print("\n\n")